# ASSIGNMENT 1 - EXCERCISE 2

### Solve a travelling salesman problem for 10 random matrices

#### The following example will be shown with full detail for a 5x5 matrix. Since the assignment requires to solve 10 different matrices, only the final result will be shown for the remaining 9 tests

In [1]:
from pulp import *

In [2]:
#Create a origin-destination matrix of the distances between the cities
#Note that the center diagonal is of 0's since the pair is origin i = destination j
#First instance will be created with 14 cities
import random
def arrange(x, rows, cols):
    random.shuffle(x)
    return [x[cols * i : cols * (i + 1)] for i in range(rows)]
origin_destination_matrix = (arrange(list(range(14*14)), 14, 14))
print(origin_destination_matrix)

[[1, 132, 108, 148, 185, 166, 171, 89, 58, 59, 24, 145, 37, 188], [2, 97, 114, 18, 77, 184, 100, 8, 53, 87, 101, 165, 117, 102], [181, 111, 107, 28, 29, 41, 0, 42, 80, 195, 96, 191, 176, 71], [125, 93, 39, 131, 177, 124, 55, 174, 121, 34, 35, 51, 110, 88], [173, 11, 52, 61, 26, 163, 44, 15, 136, 66, 67, 120, 172, 5], [92, 153, 50, 118, 46, 178, 81, 82, 85, 12, 126, 103, 112, 69], [22, 105, 36, 168, 146, 143, 10, 94, 159, 31, 19, 134, 162, 78], [193, 86, 156, 49, 7, 115, 64, 3, 140, 30, 13, 141, 169, 84], [109, 74, 43, 160, 167, 47, 113, 63, 99, 152, 142, 14, 45, 151], [72, 122, 192, 138, 144, 70, 60, 104, 133, 175, 95, 57, 158, 157], [16, 194, 23, 182, 106, 90, 4, 183, 190, 164, 128, 20, 186, 189], [32, 149, 68, 139, 21, 54, 98, 65, 180, 170, 48, 62, 76, 147], [73, 137, 25, 123, 33, 130, 187, 9, 119, 79, 154, 27, 135, 155], [150, 40, 91, 116, 161, 127, 56, 6, 17, 129, 179, 75, 38, 83]]


In [3]:
#Create variables with the parameters of the matrix to work with
cities = len(origin_destination_matrix)
indexes = range(cities)
print("Cities: ", cities)
print("Indexes: ", indexes)

Cities:  14
Indexes:  range(0, 14)


In [4]:
#Create a list of all the coordinates combinations of the matrix, excluding the center diagonal of 0's
E = [(i,j) for i in indexes for j in indexes if i!=j]

In [5]:
#Instruction for minimizing the problem
m = LpProblem("TSP",LpMinimize)

In [6]:
#Create a dictionary of all coordinate pairs
#Associate each pair with an X to assign 0 or 1 afterwards if the city will be chosen as part of the optimal route
x = LpVariable.dicts('x',E,cat=LpBinary)

In [7]:
#Create the objective function of costs per each pair
#The costs are the distances stated in the origin-destination matrix, excluding the diagonal
m += sum([origin_destination_matrix[i][j]*x[(i,j)]for (i,j) in E])

In [8]:
#Create the constraints
#It limits to a step size of 1, where from one origin, movement can only occur to one destination at a time
#The function calculates each pair both back and forth 
for i in indexes:
    m += sum([x[(i,j)] for j in indexes if i!=j]) == 1, "One out of "+str(i)
    m += sum([x[(j,i)] for j in indexes if i!=j]) == 1, "One in to "+str(i)

In [9]:
#Instructions for preliminar solution of the system without adding the restriction of subtours
m.solve()
vobj = value(m.objective)
xsol = {e:value(x[e]) for e in E}
print("- Optimal routes without eliminating subtours:")
sol_summary={x for x,y in xsol.items() if y!=0}
print("Segments within route: ", sol_summary)

- Optimal routes without eliminating subtours:
Segments within route:  {(13, 8), (10, 2), (4, 13), (12, 7), (3, 1), (7, 10), (0, 12), (9, 5), (2, 6), (6, 0), (5, 9), (1, 3), (11, 4), (8, 11)}
Tour length: 354


In [10]:
#Find successive cities that have the shortest path between them
def subtour(xsol):
    succ = 0
    subt = [succ]
    while True:
        succ = sum(xsol[succ,j]*j for j in indexes if j!=succ)
        if succ == 0: break
        subt.append(int(succ))
    return subt

In [11]:
#Iterate the previous definition to find all possible short subtours
#The function will add more nodes as it runs out of shorter routes
while True:
    subt = subtour(xsol)
    if len(subt) == cities:
        print("- Optimal tour excluding subtours")
        print("Optimal tour found: %r"%subt)
        print("Optimal tour length: %g"%vobj)
        break
    print("- Identify subtours")
    print("Subtour found: %r"%subt)
    m += sum([x[(i,j)] for i in subt for j in subt if i!=j])<= len(subt)-1
    m.solve()
    vobj = value(m.objective)
    xsol = {e: value(x[e]) for e in E}

- Identify subtours
Subtour found: [0, 12, 7, 10, 2, 6]
- Identify subtours
Subtour found: [0, 12, 7, 10, 6, 13, 8, 11, 4, 1]
- Identify subtours
Subtour found: [0, 12, 7, 10]
- Identify subtours
Subtour found: [0, 12, 7, 4, 13, 8, 11]
- Identify subtours
Subtour found: [0, 12, 7, 10, 11, 4, 13, 8, 1, 3, 2, 6]
- Identify subtours
Subtour found: [0, 12, 7, 4, 1, 3, 9, 5, 13, 8, 11]
- Identify subtours
Subtour found: [0, 12, 7, 4, 13, 8, 1]
- Identify subtours
Subtour found: [0, 12, 7, 4, 13, 8, 1, 3, 2, 6, 10]
- Identify subtours
Subtour found: [0, 12, 7, 10, 6, 9, 5, 13, 8, 11, 4, 1]
- Optimal tour excluding subtours
Optimal tour found: [0, 12, 7, 10, 6, 2, 3, 9, 5, 13, 8, 11, 4, 1]
Optimal tour length: 365


#### Solution for matrices from 5x5 to 13x13

In [12]:
def arrange(x, rows, cols):
    random.shuffle(x)
    return [x[cols * i : cols * (i + 1)] for i in range(rows)]
def subtour(xsol):
    succ = 0
    subt = [succ]
    while True:
        succ = sum(xsol[succ,j]*j for j in indexes if j!=succ)
        if succ == 0: break
        subt.append(int(succ))
    return subt
for index in range(5,14):
    print("===================================================================================================================")
    print("Matrix = ", index, "x", index)
    origin_destination_matrix = (arrange(list(range(index*index)), index, index))
    cities = len(origin_destination_matrix)
    indexes = range(cities)
    E = [(i,j) for i in indexes for j in indexes if i!=j]
    m = LpProblem("TSP",LpMinimize)
    x = LpVariable.dicts('x',E,cat=LpBinary)
    m += sum([origin_destination_matrix[i][j]*x[(i,j)]for (i,j) in E])
    for i in indexes:
        m += sum([x[(i,j)] for j in indexes if i!=j]) == 1, "One out of "+str(i)
        m += sum([x[(j,i)] for j in indexes if i!=j]) == 1, "One in to "+str(i)
    m.solve()
    vobj = value(m.objective)
    xsol = {e:value(x[e]) for e in E}
    print("- Optimal routes without eliminating subtours:")
    sol_summary={x for x,y in xsol.items() if y!=0}
    print("Segments within route: ", sol_summary)

    while True:
        subt = subtour(xsol)
        if len(subt) == cities:
            print("- Optimal tour excluding subtours")
            print("Optimal tour found: %r"%subt)
            print("Optimal tour length: %g"%vobj)
            break
        print("- Identify subtours")
        print("Subtour found: %r"%subt)
        m += sum([x[(i,j)] for i in subt for j in subt if i!=j])<= len(subt)-1
        m.solve()
        vobj = value(m.objective)
        xsol = {e: value(x[e]) for e in E}

Matrix =  5 x 5
- Optimal routes without eliminating subtours:
Segments within route:  {(3, 4), (4, 1), (2, 0), (0, 2), (1, 3)}
- Identify subtours
Subtour found: [0, 2]
- Optimal tour excluding subtours
Optimal tour found: [0, 4, 1, 3, 2]
Optimal tour length: 40
Matrix =  6 x 6
- Optimal routes without eliminating subtours:
Segments within route:  {(3, 1), (5, 4), (4, 2), (2, 3), (0, 5), (1, 0)}
- Optimal tour excluding subtours
Optimal tour found: [0, 5, 4, 2, 3, 1]
Optimal tour length: 29
Matrix =  7 x 7
- Optimal routes without eliminating subtours:
Segments within route:  {(6, 2), (2, 1), (3, 4), (4, 3), (1, 5), (0, 6), (5, 0)}
- Identify subtours
Subtour found: [0, 6, 2, 1, 5]
- Optimal tour excluding subtours
Optimal tour found: [0, 2, 1, 5, 6, 4, 3]
Optimal tour length: 57
Matrix =  8 x 8
- Optimal routes without eliminating subtours:
Segments within route:  {(2, 1), (6, 4), (5, 7), (7, 3), (4, 5), (0, 2), (3, 6), (1, 0)}
- Identify subtours
Subtour found: [0, 2, 1]
- Optimal t